# Setup

In [1]:
import contextily as cx
import figure_utilities
import constants
from stats_utilities import produce_summary_statistics, select_controls, test_balance
import geopandas as gpd
import matplotlib.pyplot as plt
from panel_utilities import get_value_variable_names, prepare_df_for_DiD
import numpy as np
plt.rcParams['savefig.dpi'] = 300
import os
from differences import ATTgt

import pandas as pd

/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_28117/3958762526.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/lates

In [2]:
# Store paths.
INPUT_DATA_PANEL = "../data/03_cleaned/crime_analysis_monthly.parquet"
OUTPUT_TABLES = "../output/final_paper/tables"
OUTPUT_FIGURES = "../output/final_paper/figures"

# Main Results

In [3]:
# Read fresh copy of unrestricted dataset into memory.
df = pd.read_parquet(INPUT_DATA_PANEL)
treatment_date_variable = 'latest_docket_month'  # Store treatment date variable.


In [4]:
# Generate value variables list and dictionaries mapping between months and integers.
analysis = f"group_0_crimes_{constants.Analysis.MAIN_RESULTS_RADIUS}m"
weekly_value_vars_crime, month_to_int_dictionary, int_to_month_dictionary = get_value_variable_names(df, analysis)

In [5]:
# Test for Confounders
covariates_exploration_df = select_controls(df=df, analysis=analysis,
                                            treatment_date_variable=treatment_date_variable,
                                            output_directory=OUTPUT_TABLES)
covariates_exploration_df


Dependent Variable  \
                                                                                                    Change in Crime Incidents, April 2019-March 2020   
                                                 \emph{Independent Variable}                                                                           
Panel A: Pre-treatment Outcomes                  total_twenty_seventeen_group_0_crimes_250m                                             4.156413e-01   
                                                 pre_treatment_change_in_group_0_crimes_250m                                           5.568113e-149   
                                                 relative_pre_treatment_change_in_group_0_crimes...                                    7.324643e-255   
Panel B: Census Tract Characteristics            frac_coll_plus2010                                                                     9.648084e-01   
                                                 job_density_2013                                                                       8.721894e-08   
                                                 med_hhinc2016                                                                          3.649106e-01   
                                                 poor_share2010                                                                         4.202086e-02   
                                                 popdensity2010                                                                         7.874144e-05   
                                                 share_white2010                                                                        4.545321e-02   
Panel C: Case Initiation                         for_cause                                                                              9.352558e-01   
                                                 no_cause                                                                               3.040212e-01   
                                                 non_payment                                                                            2.762685e-01   
Panel D: Defendant and Plaintiff Characteristics hasAttyD                                                                               9.781505e-01   
                                                 hasAttyP                                                                               1.604738e-01   
                                                 isEntityD                                                                              6.567879e-01   
                                                 isEntityP                                                                              4.028115e-04   

                                                                                                                      
                                                                                                    Treated Property  
                                                 \emph{Independent Variable}                                          
Panel A: Pre-treatment Outcomes                  total_twenty_seventeen_group_0_crimes_250m             4.017521e-01  
                                                 pre_treatment_change_in_group_0_crimes_250m            6.851633e-03  
                                                 relative_pre_treatment_change_in_group_0_crimes...     1.959620e-01  
Panel B: Census Tract Characteristics            frac_coll_plus2010                                     8.133641e-01  
                                                 job_density_2013                                       5.501121e-01  
                                                 med_hhinc2016                                          1.427126e-02  
                                                 poor_share2010                                         4.904136e-03  
                                                 po

In [6]:
# Re-Balance on Controls


balance_table, pre_treatment_covariates = test_balance(df, analysis, covariates_exploration_df, OUTPUT_TABLES)
balance_table


\
                                                           Cases Won by Plaintiff   
Panel A Total Crime Incidents, 2017                                    338.181152   
        Change in Crime Incidents, 2017-2019                           -50.048168   
        relative_pre_treatment_change_in_group_0_crimes...              -4.663874   
Panel B Bachelor's degree, 2010                                          0.316407   
        Job density, 2013                                            16161.032515   
        Median household income, 2016                                47553.063874   
        Poverty rate, 2010                                               0.279638   
        Population density, 2010                                     23320.185868   
        Share white, 2010                                                0.319421   
Panel C Filing for cause                                                 0.132984   
        Filing without cause                                             0.033508   
        Filing for nonpayment                                            0.716230   
Panel D Defendant has attorney                                           0.011518   
        Plaintiff has attorney                                           0.884817   
        Defendant is entity                                              0.019895   
        Plaintiff is entity                                              0.808377   

                                                           Difference in Cases Won by Defendant  \
                                                                                     Unweighted   
Panel A Total Crime Incidents, 2017                                                  -10.547731   
        Change in Crime Incidents, 2017-2019                                         -15.579503   
        relative_pre_treatment_change_in_group_0_crimes...                            -1.688398   
Panel B Bachelor's degree, 2010                                                        0.002512   
        Job density, 2013                                                          -1272.661768   
        Median household income, 2016                                               2997.870414   
        Poverty rate, 2010                                                            -0.021030   
        Population density, 2010                                                    -297.766413   
        Share white, 2010                                                              0.012446   
Panel C Filing for cause                                                               0.043066   
        Filing without cause                                                          -0.020988   
        Filing for nonpayment                                                         -0.106658   
Panel D Defendant has attorney                                                        -0.048427   
        Plaintiff has attorney                                                        -0.049788   
        Defendant is entity                                                           -0.008715   
        Plaintiff is entity                                                           -0.077182   

                                                                          \
                                                                \emph{p}   
Panel A Total Crime Incidents, 2017                         4.017521e-01   
        Change in Crime Incidents, 2017-2019                6.851633e-03   
        relative_pre_treatment_change_in_group_0_crimes...  1.959620e-01   
Panel B Bachelor's degree, 2010                             8.133641e-01   
        Job density, 2013                                   5.501121e-01   
        Median household income, 2016                       1.427126e-02   
        Poverty rate, 2010                                  4.904136e-03   
        Population density, 2010                            6.737331e-01   
        S

In [7]:
# Prepare df for use with the differences package.
df = df.reset_index()
df = prepare_df_for_DiD(df=df,
                        analysis=analysis,
                        treatment_date_variable=treatment_date_variable,
                        pre_treatment_covariates=pre_treatment_covariates,
                        missing_indicators=[],
                        value_vars=weekly_value_vars_crime,
                        period_to_int_dictionary=month_to_int_dictionary)

# Run DiD conditional on covariates.
att_gt_all_crimes = ATTgt(data=df, cohort_name=treatment_date_variable, base_period='universal')
formula = f'{analysis} ~ ' + '+'.join(pre_treatment_covariates)
result = att_gt_all_crimes.fit(formula=formula, control_group='never_treated', n_jobs=-1, progress_bar=True)

# Plot D.R. ATT(t-g)s on a long horizon.
fig, ax = plt.subplots(layout='constrained')
figure_utilities.aggregate_by_event_time_and_plot(att_gt_all_crimes,
                                                  start_period=constants.Analysis.MINIMUM_PRE_PERIOD,
                                                  end_period=constants.Analysis.MAXIMUM_POST_PERIOD,
                                                  title="", ax=ax)

figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "att_gt_dr_all_crimes.png"))

Computing ATTgt [workers=50]  100%|████████████████████| 1176/1176 [00:21<00:00, 55.56it/
